**Purpose of the notebook: Format crude oil and gas prices (remove taxes, convert in euros, per liter).**

In [1]:
import os
os.chdir("C:/Users/zetru/OneDrive/Mémoire python/")
import numpy as np
import pandas as pd
df=pd.read_hdf('df_v3.h5','df_v3')

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = 16, 9
plt.rcParams['axes.grid'] = True

In [3]:
df['litre_brut']=df['Prix_Brent_euros']/158.987294928

In [4]:
df.drop(columns=['Data 1: Europe Brent Spot Price FOB (Dollars per Barrel)','Prix_Brent_euros','Date'],inplace=True)

In [6]:
ticpe={
    'Gazole':0.6075,
    'SP95':0.6902,
    'SP98':0.6902,
    'E10':0.6702,
    'E85':0.1183,
    'GPLc':0.2071
}

ticpe_idf={
    'Gazole':0.6264,
    'SP95':0.7004,
    'SP98':0.7004,
    'E10':0.6804,
    'E85':0.1183,
    'GPLc':0.2071
}
tva=0.2
idf=['75','77','78','91','92','93','94','95']

In [7]:
df['valeur'] = np.where(
    df['cp'].isin(idf),
    (df['valeur'] / (1 + tva)) - df['nom_carburant'].map(ticpe_idf),
    (df['valeur'] / (1 + tva)) - df['nom_carburant'].map(ticpe)
)

In [8]:
remise15=pd.to_datetime(['2022-04-01','2022-08-31'])
remise25=pd.to_datetime(['2022-09-01','2022-11-15'])
remise25

DatetimeIndex(['2022-09-01', '2022-11-15'], dtype='datetime64[ns]', freq=None)

In [9]:
df.loc[(df['maj']>=remise15[0])&(df['maj']<=remise15[1]), 'valeur'] += 0.15
df.loc[(df['maj']>=remise25[0])&(df['maj']<=remise25[1]), 'valeur'] += 0.25
df.loc[(df['maj']>remise25[1]), 'valeur'] += 0.0833

In [10]:
valeurs_aberrantes=df.index[~((df['nom_carburant']=='E85')|(df['nom_carburant']=='GPLc'))&(df['valeur']<df['litre_brut'])]
df=df.drop(valeurs_aberrantes).reset_index(drop=True)

In [11]:
df_stats=pd.DataFrame(df.groupby(['nom_carburant','maj'])['valeur'].mean())
df_stats.reset_index(inplace=True)

In [13]:
df

,cp,id,nom_carburant,maj,type,latitude,longitude,code_postal,ville,id_carburant,valeur,litre_brut
0,01,1000001,Gazole,2022-01-03,R,46.201,5.198,01000,SAINT-DENIS-LèS-BOURG,1,0.702500,0.435626
1,01,1000001,Gazole,2022-01-05,R,46.201,5.198,01000,SAINT-DENIS-LèS-BOURG,1,0.692500,0.447949
2,01,1000001,Gazole,2022-01-10,R,46.201,5.198,01000,SAINT-DENIS-LèS-BOURG,1,0.728333,0.452874
3,01,1000001,Gazole,2022-01-17,R,46.201,5.198,01000,SAINT-DENIS-LèS-BOURG,1,0.739167,0.484043
4,01,1000001,Gazole,2022-01-21,R,46.201,5.198,01000,SAINT-DENIS-LèS-BOURG,1,0.769167,0.497616
...,...,...,...,...,...,...,...,...,...,...,...,...
4537444,95,95870010,SP98,2022-12-23,R,48.936,2.206,95870,Bezons,6,0.856233,0.488413
4537445,95,95870010,SP98,2022-12-27,R,48.936,2.206,95870,Bezons,6,0.874567,0.487324
4537446,95,95870010,SP98,2022-12-28,R,48.936,2.206,95870,Bezons,6,0.874567,0.484175
4537447,95,95870010,SP98,2022-12-29,R,48.936,2.206,95870,Bezons,6,0.879567,0.477600


In [14]:
col_temp=df.pop(df.columns[-1])
df.insert(10,col_temp.name,col_temp)

In [15]:
col_temp=df.pop(df.columns[-1])
df.insert(4,col_temp.name,col_temp)

In [16]:
df.drop('code_postal',axis=1,inplace=True)

In [17]:
df_temp=df.pop('litre_brut')
df.insert(5,'litre_brut',df_temp)

In [18]:
df_temp=df.pop('id_carburant')
df.insert(3,'id_carburant',df_temp)

In [19]:
df

,cp,id,nom_carburant,id_carburant,maj,valeur,litre_brut,type,latitude,longitude,ville
0,01,1000001,Gazole,1,2022-01-03,0.702500,0.435626,R,46.201,5.198,SAINT-DENIS-LèS-BOURG
1,01,1000001,Gazole,1,2022-01-05,0.692500,0.447949,R,46.201,5.198,SAINT-DENIS-LèS-BOURG
2,01,1000001,Gazole,1,2022-01-10,0.728333,0.452874,R,46.201,5.198,SAINT-DENIS-LèS-BOURG
3,01,1000001,Gazole,1,2022-01-17,0.739167,0.484043,R,46.201,5.198,SAINT-DENIS-LèS-BOURG
4,01,1000001,Gazole,1,2022-01-21,0.769167,0.497616,R,46.201,5.198,SAINT-DENIS-LèS-BOURG
...,...,...,...,...,...,...,...,...,...,...,...
4537444,95,95870010,SP98,6,2022-12-23,0.856233,0.488413,R,48.936,2.206,Bezons
4537445,95,95870010,SP98,6,2022-12-27,0.874567,0.487324,R,48.936,2.206,Bezons
4537446,95,95870010,SP98,6,2022-12-28,0.874567,0.484175,R,48.936,2.206,Bezons
4537447,95,95870010,SP98,6,2022-12-29,0.879567,0.477600,R,48.936,2.206,Bezons


In [20]:
df['id'].value_counts()

id
33167001    2036
65429001    1842
53200005    1825
44460001    1825
56700004    1825
            ... 
53100006       3
38510004       2
43200008       2
37400008       2
78180012       1
Name: count, Length: 9470, dtype: int64

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4537449 entries, 0 to 4537448
Data columns (total 11 columns):
 #   Column         Dtype         
---  ------         -----         
 0   cp             object        
 1   id             object        
 2   nom_carburant  object        
 3   id_carburant   object        
 4   maj            datetime64[ns]
 5   valeur         float64       
 6   litre_brut     float64       
 7   type           object        
 8   latitude       float64       
 9   longitude      float64       
 10  ville          object        
dtypes: datetime64[ns](1), float64(4), object(6)
memory usage: 380.8+ MB


In [23]:
df.to_hdf('df_v4.h5',key='df_v4',mode='w')